In [ ]:
# Install packages psycopg2-binary boto3

!pip install psycopg2-binary boto3

In [ ]:
# HOST = "" # Fill this
# DATA_BUCKET = "" # Fill this 
DATABASE = "your database name"
USER = "Your master username"
PASSWORD = "your database password"
PORT = 5439

In [ ]:
# Establish a connection to the database and test that connection by reading the database name

import psycopg2
from pprint import pprint

query = '''SELECT datname FROM pg_database;'''

conn = psycopg2.connect(
    host=HOST,
    user=USER,
    port=PORT,
    password=PASSWORD,
    dbname=DATABASE
)

def runquery(conn,query,commit_bool=False):
    """
    Just run a query given a connection
    """
    
    curr=conn.cursor()
    curr.execute(query)
    if commit_bool:
        conn.commit()
        return None
    for row in curr.fetchall():
        pprint(row)
    return None

runquery(conn, query)

In [ ]:
# Create a redshift table

table_create_query = '''
create table movies(
    title varchar(300) not null,
    year integer not null,
    rating real not null,
    running_time_secs integer not null
);
'''

runquery(conn, table_create_query, commit_bool=True)


In [ ]:
# Check if the databse was created or not?

view_public_tables_query = '''
SELECT DISTINCT
  tablename
FROM
  PG_TABLE_DEF
WHERE
  schemaname = 'public';
'''


runquery(conn, view_public_tables_query)

In [ ]:
# Check the info of S3
!aws s3 ls

In [ ]:
# Compress csv file to csv.gzip file for smoothly running

import gzip
import shutil
with open('./data.csv', 'rb') as f_in:
    with gzip.open('data.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

import boto3
s3 = boto3.resource('s3')
s3.meta.client.upload_file('./data.csv.gz', DATA_BUCKET, 'data.csv.gz')

In [ ]:
IAM_ROLE = "your IAM role endpoint" # fill this

In [ ]:
# Load all dataset to Redshift

copy_query = '''
copy movies from 's3://{0}/data.csv.gz'
iam_role '{1}'
CSV
GZIP
IGNOREHEADER 1;
'''.format(DATA_BUCKET, IAM_ROLE)


runquery(conn, copy_query, commit_bool=True)

In [ ]:
# Get the needed information based on requirement

best_2013_movies_query = '''
SELECT title, rating FROM movies
WHERE year = 2013 and rating > 8.0
ORDER BY rating DESC;
'''.format(DATA_BUCKET, IAM_ROLE)


runquery(conn, best_2013_movies_query)

#### Now that we know this works we can unload the data to S3 so that it can be reviewed later

In [ ]:
unload_best_2013_movies_query = '''
UNLOAD ('
    SELECT title, rating FROM movies
    WHERE year = 2013 and rating > 8.0
    ORDER BY rating DESC;'
)
TO 's3://{0}/output/'
iam_role '{1}'
'''.format(DATA_BUCKET, IAM_ROLE)


runquery(conn, unload_best_2013_movies_query)

#### Here's a nifty little query that can fix things sometimes if you edit the queries and they fail.

In [ ]:
rollback_query = '''rollback;'''
runquery(conn, rollback_query, commit_bool=True)

# All Done!